In [380]:
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB 
import pandas as pd

Считаем train данные.

In [1]:
with open('products_sentiment_train.tsv','r') as f:
    train=f.readlines()

Очистим их от служебных символов, пунктуаций и отделим зависимую переменную

In [3]:
train=[x.strip() for x in train]

In [51]:
y=[int(x.split('\t')[1]) for x in train]

In [320]:
train_t=[x.split('\t')[0] for x in train]

In [321]:
train_t=[re.sub(' +',' ',re.sub('[^a-zA-Z\d\$\!\']',' ',x)).lower() for x in train_t]

Создадим переменную число восклицательных знаков

In [324]:
exclamation=map(lambda comment: comment.count('!'),train_t)

Создадим переменную число уникальных слов

In [342]:
unique_wrds=map(lambda comment: len(set(w for w in comment.split())),train_t)

Создадим переменную длина отзыва

In [345]:
len_text=np.array([len(x) for x in train_t])

In [346]:
len_text=np.column_stack([np.array(len_text),np.array(exclamation),np.array(unique_wrds)])

Преобразуем переменную у в массив и выведем summary. Также сделаем pipeline для дальнейшей работы.

In [348]:
y=np.array(y)

In [349]:
print "Доля положительных отзывов: %s"%y.mean()

Доля положительных отзывов: 0.637


In [226]:
def workontext(vectorizer,classifier):
    return Pipeline([
        ("vectorizer",vectorizer),
        ("classifier",classifier)
    ])

Попробуем различные варианты 

In [227]:
cv1=cross_val_score(workontext(CountVectorizer(),LogisticRegression()),train_t,y,cv=5)

In [228]:
cv1.mean()

0.7684956843480272

In [229]:
cv2=cross_val_score(workontext(TfidfVectorizer(),LogisticRegression()),train_t,y,cv=5)

In [230]:
cv2.mean()

0.7665031843949025

In [231]:
cv3=cross_val_score(workontext(CountVectorizer(ngram_range=(1,2),stop_words='english'),LogisticRegression()),train_t,y,cv=5)

In [232]:
cv3.mean()

0.7460069062931642

In [234]:
cv4=cross_val_score(workontext(CountVectorizer(ngram_range=(1,2)),LogisticRegression()),train_t,y,cv=5)

In [235]:
cv4.mean()

0.7705044437777736

In [236]:
cv5=cross_val_score(workontext(CountVectorizer(ngram_range=(3,6),analyzer="char_wb"),LogisticRegression()),train_t,y,cv=5)

In [237]:
cv5.mean()

0.7599943874649217

In [300]:
cv6=cross_val_score(workontext(TfidfVectorizer(ngram_range=(2,6),analyzer="char_wb",min_df=3),\
                               LogisticRegression()),train_t,y,cv=5)

In [301]:
cv6.mean()

0.7814982031137695

In [304]:
cv7=cross_val_score(workontext(TfidfVectorizer(),LogisticRegression()),train_t,y,cv=5)

In [305]:
cv7.mean()

0.7665031843949025

Разделим выборку на train и test. Рассчитаем результаты на train выборке и скомбинируем результаты для методов из cv4, cv5,cv6,cv7.

In [350]:
wrk1=workontext(CountVectorizer(ngram_range=(1,2)),LogisticRegression())
wrk2=workontext(CountVectorizer(ngram_range=(3,6),analyzer="char_wb"),LogisticRegression())
wrk3=workontext(TfidfVectorizer(ngram_range=(2,6),analyzer="char_wb",min_df=3),LogisticRegression())
wrk4=workontext(TfidfVectorizer(),LogisticRegression())

In [351]:
train_t_train,train_t_test,y_train,y_test,len_text_train,len_text_test=\
train_test_split(train_t,y,len_text,test_size=0.3,random_state=2)

In [352]:
wrk1.fit(train_t_train,y_train)
wrk2.fit(train_t_train,y_train)
wrk3.fit(train_t_train,y_train)
wrk4.fit(train_t_train,y_train)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_id...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [353]:
pred1=wrk1.predict_proba(train_t_train)[:,1]
pred2=wrk2.predict_proba(train_t_train)[:,1]
pred3=wrk3.predict_proba(train_t_train)[:,1]
pred4=wrk4.predict_proba(train_t_train)[:,1]

Скомбинируем результаты и добавим длину отзыва. Применим random forest для общей классификации.

In [354]:
X_train=np.column_stack([pred1,pred2,pred3,pred4,len_text_train])

In [334]:
cv_tree=cross_val_score(RandomForestClassifier(n_estimators=1000,n_jobs=-1),X_train,y_train,cv=5)

In [335]:
cv_tree.mean()

1.0

In [355]:
rf1=RandomForestClassifier(n_estimators=10000,n_jobs=-1)

In [356]:
rf1.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

Применим получившийся классификатор на тестовой выборке, посмотрим что получилось.

In [357]:
pred1_test=wrk1.predict_proba(train_t_test)[:,1]
pred2_test=wrk2.predict_proba(train_t_test)[:,1]
pred3_test=wrk3.predict_proba(train_t_test)[:,1]
pred4_test=wrk4.predict_proba(train_t_test)[:,1]

In [358]:
X_test=np.column_stack([pred1_test,pred2_test,pred3_test,pred4_test,len_text_test])

In [359]:
y_pred=rf1.predict(X_test)

In [360]:
accuracy_score(y_test,y_pred)

0.7966666666666666

# Test

Возьмём тестовую выборку из файла и преобразуем её аналогичным образом.

In [367]:
with open('products_sentiment_test.tsv','r') as f:
    test=f.readlines()

In [368]:
test=test[1:]

In [369]:
ids=[x.split('\t')[0] for x in test]

In [371]:
test=[x.strip() for x in test]
test_t=[x.split('\t')[1] for x in test]
test_t=[re.sub(' +',' ',re.sub('[^a-zA-Z\d\$\!\']',' ',x)).lower() for x in test_t]
exclamation_test=map(lambda comment: comment.count('!'),test_t)
unique_wrds_test=map(lambda comment: len(set(w for w in comment.split())),test_t)
len_text_test=np.array([len(x) for x in test_t])
len_text_test=np.column_stack([np.array(len_text_test),np.array(exclamation_test),np.array(unique_wrds_test)])

Теперь повторим полное обучение классификатора на всей выборке. Затем будем рассчитывать финальные результаты для отправки на kaggle.

In [361]:
wrk1.fit(train_t,y)
wrk2.fit(train_t,y)
wrk3.fit(train_t,y)
wrk4.fit(train_t,y)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_id...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [372]:
pred1=wrk1.predict_proba(train_t)
pred2=wrk2.predict_proba(train_t)
pred3=wrk3.predict_proba(train_t)
pred4=wrk4.predict_proba(train_t)

In [373]:
X=np.column_stack([pred1,pred2,pred3,pred4,len_text])

In [374]:
forest=RandomForestClassifier(n_estimators=10000,n_jobs=-1,random_state=10)

In [375]:
forest.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10000, n_jobs=-1,
            oob_score=False, random_state=10, verbose=0, warm_start=False)

Теперь применим классификатор для тестовой выборки.

In [376]:
pred1_ttest=wrk1.predict_proba(test_t)
pred2_ttest=wrk2.predict_proba(test_t)
pred3_ttest=wrk3.predict_proba(test_t)
pred4_ttest=wrk4.predict_proba(test_t)

In [377]:
X_test=np.column_stack([pred1_ttest,pred2_ttest,pred3_ttest,pred4_ttest,len_text_test])

In [379]:
y_predict=forest.predict(X_test)

In [381]:
result=pd.DataFrame(ids)

In [385]:
result['y']=y_predict

In [387]:
result.columns=['Id','y']

In [390]:
result.to_csv('results.csv',sep=',',index=None)